In [3]:
import requests
import csv
import numpy as np

# 1. World 1

The first dataset is downloaded from 
"https://www.sciencedirect.com/science/article/pii/S1093326317306575#ec-research-data" (effected by May 2022).
World $1$ has a signal space of size $5$ and a binary ground truth, say $\{1,2\}$.  Agents are asked to grade the synthetic accessibility of compounds with scores $1$ to $5$, where $1$ indicates inappropriate to be synthesized and $5$ stands for appropriate. Scores in between lower the confidence of the grading. The binary ground truth indicates whether a compound is appropriate or inappropriate. The dataset includes the assessments of $100$ compounds (tasks) from $18$ agents, among which we only use the $62$ tasks that have the ground truth and the $5$ agents who are the experts.

In [33]:
# Read the question dataset and learn the ground truth mapping, as well as the prior of the ground truth
file_q = open('Data/W1_crowd-synthetic-accessibility-dataset/questions.csv', "r")
csv_reader = csv.reader(file_q)
ground_truth = {}
for row in csv_reader:
    if row[0] != 'problem_id':
        if row[2] == 'inappropriate':
            ground_truth[row[0]] = 1
        elif row[2] == 'appropriate':
            ground_truth[row[0]] = 2

n1 = np.count_nonzero(np.array(list(ground_truth.values())) == 1)
n2 = np.count_nonzero(np.array(list(ground_truth.values())) == 2)
w = np.array([n1,n2])/(n1+n2)
print('Ground truth prior: w =', w)

Ground truth prior: w = [0.61290323 0.38709677]


In [34]:
# Read the response dataset and collect the response from the experts and learn the confusion matrix
file_r = open('Data/W1_crowd-synthetic-accessibility-dataset/response.csv', "r")
csv_reader = csv.reader(file_r)
count = np.zeros((2,5))
Gamma = np.zeros((2,5))
for row in csv_reader:
    if row[0][0] == 'E' and row[1] in ground_truth.keys(): # if the task is graded by the expert and has ground truth
        count[ground_truth[row[1]]-1, int(row[4])-1] += 1
for i in range(2):
    Gamma[i] = count[i]/np.sum(count[i])
print('Confusion matrix: Gamma =')
print(Gamma)

Confusion matrix: Gamma =
[[0.68421053 0.22105263 0.03157895 0.03684211 0.02631579]
 [0.09166667 0.19166667 0.05       0.2        0.46666667]]


# 2. World 2

World $2$ has an identical signal space and ground truth space of size $4$ (actually the size is $5$, but we ignore the rarest one which occurs $9$ out of $300$ times). The dataset contains $6000$ classifications of the sentiment of $300$ tweets (tasks) provided by $110$ workers.

In [35]:
# Download the dataset
csv_url = 'https://eprints.soton.ac.uk/376543/1/WeatherSentiment_amt.csv'

req = requests.get(csv_url)
url_content = req.content
csv_file = open('Data/W2_weather_sentiment.csv', 'wb')
csv_file.write(url_content)
csv_file.close()

# Read the dataset
file = open('Data/W2_weather_sentiment.csv', "r")
csv_reader = csv.reader(file)
lists_from_csv = []
for row in csv_reader:
    lists_from_csv.append(row)

In [36]:
# Count the frequency of the reports
n = 110
m = 300

Workers = [] # names of workers
Tasks = [] # names of tasks
R = np.zeros((n,m))
count = np.zeros((n,m))
Y = np.zeros(m)
for data in lists_from_csv:
    if data[0] not in Workers:
        Workers.append(data[0])
    if data[1] not in Tasks:
        Tasks.append(data[1])
    i = Workers.index(data[0])
    j = Tasks.index(data[1])
    R[i,j] = int(data[2])+1
    Y[j] = int(data[3])+1
    count[i,j] = int(data[4])

In [39]:
# ignore the minority case
R[np.where(R==5)] = 0
Y[np.where(Y==5)] = 0

w = np.zeros(4)
Gamma = np.zeros((4,4))

for s in range(4):
    w[s] = np.count_nonzero(Y == s+1)/np.count_nonzero(Y != 0)

for s in range(4):
    for d in range(4):
        Gamma[s,d] = np.count_nonzero(R[:,np.where(Y == s+1)] == d+1)/np.count_nonzero(R[:,np.where(Y == s+1)] != 0)
        
print('Ground truth prior: w =', w)
print('Confusion matrix: Gamma =')
print(Gamma)

Ground truth prior: w = [0.19587629 0.24054983 0.24742268 0.3161512 ]
Confusion matrix: Gamma =
[[0.77056673 0.12157221 0.08409506 0.023766  ]
 [0.09083969 0.7351145  0.12977099 0.04427481]
 [0.03326256 0.06157113 0.86624204 0.03892427]
 [0.06785509 0.16388729 0.09890742 0.6693502 ]]
